In [1]:
# 参考 https://colab.research.google.com/github/TadaoYamaoka/ShogiAIBook2/blob/main/notebook/train.ipynb#scrollTo=zAT8jrKysnZS
# dlshogiの評価関数モデルをCore ML形式に変換
# バッチサイズNのモデルを出力する。Nは定数。

In [2]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime
!date

Sat Jan 29 08:11:10 JST 2022


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls -l "/content/drive/My Drive/ShogiAIBookData/"

total 9932211
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-001.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-002.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-003.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-004.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-005.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-006.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-007.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-008.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-009.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-010.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-011.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-012.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-013.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlsh

In [5]:
!pip install dlshogi==0.1.1

     |████████████████████████████████| 3.3 MB 8.3 MB/s 


In [6]:
!pip install coremltools

     |████████████████████████████████| 1.6 MB 7.3 MB/s 


In [7]:
%cd /content/ShogiAIBook

[Errno 2] No such file or directory: '/content/ShogiAIBook'
/content


In [8]:
!ln -s "/content/drive/My Drive/ShogiAIBook/log"   ./log
!ln -s "/content/drive/My Drive/ShogiAIBook/model" ./model
!ln -s "/content/drive/My Drive/ShogiAIBookData"   ./data

In [9]:
!cp ./data/model/model_resnet10_swish-072 ./model/

In [10]:
import torch
from dlshogi.common import *
from dlshogi.network.policy_value_network import policy_value_network
from dlshogi import serializers
from dlshogi import cppshogi

In [11]:
model = policy_value_network("resnet10_swish", add_sigmoid=True)

In [12]:
model.set_swish(False)#swishをx*sigmoid(x)で計算するモード

In [13]:
serializers.load_npz("./model/model_resnet10_swish-072", model, False)

In [14]:
model.eval()

PolicyValueNetworkAddSigmoid(
  (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l10): Conv2d(192, 192, k

In [15]:
class SingleInputModel(torch.nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model
  def forward(self, x):
    return self.model(x[:, 0:62, :, :], x[:, 62:119, :, :])

In [16]:
si_model = SingleInputModel(model)
si_model.eval()

SingleInputModel(
  (model): PolicyValueNetworkAddSigmoid(
    (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padd

In [17]:
batch_size = 64 # ここでバッチサイズを指定

In [22]:
traced_model = torch.jit.trace(si_model, torch.zeros(batch_size, 119, 9, 9))

In [23]:
traced_model

SingleInputModel(
  original_name=SingleInputModel
  (model): PolicyValueNetworkAddSigmoid(
    original_name=PolicyValueNetworkAddSigmoid
    (l1_1_1): Conv2d(original_name=Conv2d)
    (l1_1_2): Conv2d(original_name=Conv2d)
    (l1_2): Conv2d(original_name=Conv2d)
    (l2): Conv2d(original_name=Conv2d)
    (l3): Conv2d(original_name=Conv2d)
    (l4): Conv2d(original_name=Conv2d)
    (l5): Conv2d(original_name=Conv2d)
    (l6): Conv2d(original_name=Conv2d)
    (l7): Conv2d(original_name=Conv2d)
    (l8): Conv2d(original_name=Conv2d)
    (l9): Conv2d(original_name=Conv2d)
    (l10): Conv2d(original_name=Conv2d)
    (l11): Conv2d(original_name=Conv2d)
    (l12): Conv2d(original_name=Conv2d)
    (l13): Conv2d(original_name=Conv2d)
    (l14): Conv2d(original_name=Conv2d)
    (l15): Conv2d(original_name=Conv2d)
    (l16): Conv2d(original_name=Conv2d)
    (l17): Conv2d(original_name=Conv2d)
    (l18): Conv2d(original_name=Conv2d)
    (l19): Conv2d(original_name=Conv2d)
    (l20): Conv2d(orig

In [24]:
import coremltools as ct

In [27]:
ct_input_x = ct.TensorType(name='x', shape=(batch_size, 119, 9, 9))

In [28]:
mlmodel = ct.convert(traced_model, inputs=[ct_input_x])

Running MIL Common passes:   0%|          | 0/34 [00:00<?, ? passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '406', of the source model, has been renamed to 'var_406' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '429', of the source model, has been renamed to 'var_429' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 257/257 [00:00<00:00, 265.45 ops/s]


In [29]:
# macでないと実行不可
# results = mlmodel.predict({"x": x.numpy()})

In [30]:
# このままだと、出力変数の名前がvar_577のような実装依存の値になり、Swift側での受け取りに不便なので名前を固定する
spec = mlmodel.get_spec()
move_name, result_name = mlmodel.output_description
ct.utils.rename_feature(spec, move_name, "move")
ct.utils.rename_feature(spec, result_name, "result")
mlmodel = ct.models.MLModel(spec)

In [31]:
!mkdir -p ./model/batch{batch_size}

In [32]:
mlmodel.save(f"./model/batch{batch_size}/DlShogiResnet10SwishBatch.mlmodel")

In [33]:
move, game_result = si_model(torch.zeros(batch_size, 119, 9, 9))

In [34]:
move, game_result

(tensor([[-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6754, -3.4884],
         [-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6754, -3.4884],
         [-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6754, -3.4884],
         ...,
         [-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6754, -3.4884],
         [-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6754, -3.4884],
         [-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6754, -3.4884]],
        grad_fn=<AddBackward0>), tensor([[0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.6059],
         [0.60

In [35]:
move.shape

torch.Size([64, 2187])

In [36]:
game_result.shape

torch.Size([64, 1])